In [4]:
import numpy as np

In [53]:
class Sperm2D:
    def __init__(self,
                 length=70.1,
                 n_segments=50,
                 bending_modulus=1800,
                 amplitude=0.2,
                 wavenumber=1.0, # Note: not the conventional wavenumner. 2*pi*k*l/L, where k is the wavenumber
                 frequency=10.0,
                init_position=[0, 0], # tip of head
                init_angle=0,
                phase=0,
                head_semi_major=3,
                head_semi_minor=1):
        """
        Initialize a single 2D sperm filament.
        
        Parameters
        ----------
        length : float
            Total length of the filament (L).
        n_segments : int
            Number of discrete segments (N).
        bending_modulus : float
            Bending stiffness K_B.
        amplitude, wavenumber, frequency : float
            Parameters for the preferred-curvature waveform kappa(s,t).
        """
        # Geometry
        self.L = length
        self.N = n_segments
        self.Delta_L = length / n_segments
        self.a = head_semi_major
        self.b = head_semi_minor
        
        # Mechanical parameters
        self.K_B = bending_modulus
        self.K_0 = amplitude
        self.k = wavenumber
        self.omega = frequency
        self.phi = phase
        
        # State: positions Y[0..N], angles theta[0..N]
        self.Y_0 = init_position
        self.theta_0 = init_angle
        segments = np.hstack([self.a, np.arange(self.Delta_L/2+2*self.a, self.L+2*self.a, self.Delta_L)])
        self.Y = np.vstack([self.Y_0[0]+segments*np.cos(self.theta_0), self.Y_0[1]+segments*np.sin(self.theta_0)]).T # An array of length N+1 (midpoint of segmenets)
        self.theta = np.array([init_angle]*(self.N+1)) # An array of length N+1 (midpoint of segments)
        
        # Lagrange multipliers for constraints (N+2 of them), corresponding to the edges of each segment
        self.Lambda = np.zeros(self.N+2)
    
    def preferred_curvature(self, t):
        """
        Traveling-wave preferred curvature Kappa(s,t) along filament using Eq. 2.1 of Schoeller et al. 2018
        Returns an array of length N, corresponding to the midpoint of each filament segment only, i.e. no head.
        """
        s = np.arange(self.Delta_L/2, self.L, self.Delta_L)
        base = self.K_0 * np.sin(2*np.pi*self.k*s/self.L - self.omega*t)
        decay = np.where(s > self.L/2, 2*(self.L - s)/self.L, 1.0)
        return base * decay
    
    def internal_moment(self, t):
        """
        Compute M_{n+1/2} for n=1..N+1 using Eq. 34 of Schoeller et al. 2020
        Returns an array of length N+2, corresponding to the edges of each segment.
        """
        kappa = self.preferred_curvature(t)
        t_hat_x, t_hat_y = np.cos(self.theta), np.sin(self.theta)
        cross = t_hat_x[:-1]*t_hat_y[1:] - t_hat_y[:-1]*t_hat_x[1:]
        delta_s = np.zeros(self.N)
        delta_s[0] = self.a+self.Delta_L/2
        delta_s[1:] = self.Delta_L
        M = np.zeros(self.N+2)
        M[1:self.N+1] = self.K_B * (cross/delta_s - kappa)
        return M
    
    # Placeholder methods to fill in next:
    def compute_forces(self):
        """Assemble internal constraint + bending forces Fc."""
        pass
    
    def mobility(self):
        """Compute segment velocities via chosen hydrodynamic model."""
        pass
    
    def step(self, dt, Y_prev, theta_prev):
        """
        Perform one implicit BDF2 time step:
          - Form nonlinear residuals (position update, angle update, constraints)
          - Solve for Y_new, theta_new, Lambda
        """
        pass


In [54]:
sperm_1 = Sperm2D()

In [55]:
sperm_1.internal_moment(0)

array([   0.        ,  -22.60458703,  -67.45727325, -111.24611797,
       -153.28054496, -192.89764619, -229.47263631, -262.42870587,
       -291.24611797, -315.47040482, -334.71953492, -348.68993801,
       -357.16129247, -360.        , -357.16129247, -348.68993801,
       -334.71953492, -315.47040482, -291.24611797, -262.42870587,
       -229.47263631, -192.89764619, -153.28054496, -111.24611797,
        -67.45727325,  -22.60458703,   22.15249529,   63.40983686,
        100.12150618,  131.82126867,  158.17606988,  178.98865632,
        194.19724235,  203.87228258,  208.21046718,  207.52611165,
        202.24016404,  192.86709794,  180.        ,  164.29419454,
        146.44977396,  127.19342327,  107.25993764,   87.37383539,
         68.23146353,   50.48397999,   34.72157631,   21.45927629,
         11.1246118 ,    4.0474364 ,    0.45209174,    0.        ])